<a href="https://colab.research.google.com/github/Guilherme774/Data-Science-and-Machine-Learning-Projects/blob/master/TensorFlow_Introdu%C3%A7%C3%A3o_ao_sintonizador_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Configuração
import tensorflow as tf
from tensorflow import keras

import IPython

In [ ]:
!pip install -q -U keras-tuner
import kerastuner as kt

     |████████████████████████████████| 61kB 4.4MB/s 


In [ ]:
# Baixando e preparando o conjunto de dados
  # Carregando os dados do banco
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [ ]:
# Definindo o modelo
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))
  
  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer 
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
  
  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])
  
  return model

In [ ]:
# Instanciando o sintonizador e executando a sintonia elevada
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')                       

In [ ]:
# definindo um retorno de chamada para limpar as saídas de treinamento no final de cada etapa de treinamento.
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [ ]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 416 and the optimal learning rate for the optimizer
is 0.001.



In [ ]:
# Treinando o modelo com os parâmetros ideias da pesquisa
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.4842 - accuracy: 0.8282 - val_loss: 0.4245 - val_accuracy: 0.8514
Epoch 2/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3599 - accuracy: 0.8674 - val_loss: 0.3713 - val_accuracy: 0.8649
Epoch 3/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3251 - accuracy: 0.8787 - val_loss: 0.3477 - val_accuracy: 0.8727
Epoch 4/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3008 - accuracy: 0.8888 - val_loss: 0.3642 - val_accuracy: 0.8713
Epoch 5/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2817 - accuracy: 0.8955 - val_loss: 0.3490 - val_accuracy: 0.8737
Epoch 6/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2666 - accuracy: 0.9009 - val_loss: 0.3436 - val_accuracy: 0.8796
Epoch 7/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2534 - accuracy: 0.9053 - val_loss: 0.3352 - val_accuracy: